In [1]:
import autogen
from autogen import ConversableAgent

In [2]:
from utils import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()
llm_config = {"model": "gpt-5-mini","api_key":OPENAI_API_KEY}

In [3]:
from autogen import UserProxyAgent

In [4]:
import pprint

In [5]:
key_clause_extractor = ConversableAgent( name = "key_clause_extractor_Agent",
                                         system_message=""" You are an key clause extractor agent, your role is to review the legal content submitted and 
                                         extract the key clauses from the content""",
                                         llm_config=llm_config,
                                         human_input_mode="NEVER")

Risk_Flagger= ConversableAgent( name= "Risk_flagger_Agent",
                                system_message=""" You are an Risk Flagger Agent, your role is to flag any risk or harmful contents from the legal
                                contents""",
                                llm_config=llm_config,
                                human_input_mode="NEVER")

Final_reviewer= ConversableAgent( name= "Final_reviewer_Agent",
                                system_message=""" Final_reviewer_Agent, your role is to verify the entire legal content, and suggest any improvements if required 
                                and highlight any risky contents""",
                                llm_config=llm_config,
                                human_input_mode="NEVER")

In [6]:
User_proxy= UserProxyAgent( name ="UserProxyAgent",
                            human_input_mode="ALWAYS",
                            code_execution_config=False)
                              

In [7]:
cordinator= ConversableAgent( name = "cordinator_Agent",
                                 system_message="""Your role is like a mediator who gets the legal content from user proxy and pass it on to individual agents
                                 which will process the content further""",
                                 llm_config=llm_config,
                                 human_input_mode="NEVER")

In [8]:
message = """The Receiving Party agrees to treat all non-public information received
    from the Disclosing Party as strictly confidential. This includes technical data, business plans, and client lists.
    The Receiving Party shall not disclose, publish, or use this information for any purpose other than evaluating the proposed business relationship without prior written consent.
    This obligation does not apply to information that is publicly known through no fault of the Receiving Party.
    Upon termination of discussions, the Receiving Party shall return or destroy all confidential materials."""

"sender" : User_proxy,
               "recipient" : cordinator,
               "message" : """The Receiving Party agrees to treat all non-public information received
    from the Disclosing Party as strictly confidential. This includes technical data, business plans, and client lists.
    The Receiving Party shall not disclose, publish, or use this information for any purpose other than evaluating the proposed business relationship without prior written consent.
    This obligation does not apply to information that is publicly known through no fault of the Receiving Party.
    Upon termination of discussions, the Receiving Party shall return or destroy all confidential materials.""",
               "max_turns": 1,
               "summary_method": "last_msg",

               { "sender" : cordinator,
               "recipient" : User_proxy,
               "max_turns": 1,
               #"carryover": [0,1,2,3],
               "summary_method": "last_msg"
             },

In [9]:
#message= cordinator.chat_messages_for_summary(User_proxy)[-1]['content']
#message

chat_flow= [ 
             { "sender" : User_proxy,
               "recipient" : key_clause_extractor,
               "max_turns": 1,
               "message" : message,
               #"summary_method": "reflection_with_llm",
             },
             { "sender" : User_proxy,
               "recipient" : Risk_Flagger,
               "message" : message,
               "max_turns": 1,
               #"summary_method": "reflection_with_llm",
             },
             { "sender" : User_proxy,
               "recipient" : Final_reviewer,
               "message" : message,
               #"carryover": [0,1],
               "max_turns": 1,
               #"summary_method": "reflection_with_llm",
             },
             
           ]

In [10]:
from autogen import initiate_chats
chats= initiate_chats(chat_flow)


********************************************************************************
Starting a new chat....

********************************************************************************
UserProxyAgent (to key_clause_extractor_Agent):

The Receiving Party agrees to treat all non-public information received
    from the Disclosing Party as strictly confidential. This includes technical data, business plans, and client lists.
    The Receiving Party shall not disclose, publish, or use this information for any purpose other than evaluating the proposed business relationship without prior written consent.
    This obligation does not apply to information that is publicly known through no fault of the Receiving Party.
    Upon termination of discussions, the Receiving Party shall return or destroy all confidential materials.

--------------------------------------------------------------------------------
key_clause_extractor_Agent (to UserProxyAgent):

Here are the key clauses extracted f

In [18]:
User_proxy.chat_messages
# this function returns the conversation held by user_proxy agent

defaultdict(list,
            {<autogen.agentchat.conversable_agent.ConversableAgent at 0x1c3dae7b290>: [{'content': 'The Receiving Party agrees to treat all non-public information received\n    from the Disclosing Party as strictly confidential. This includes technical data, business plans, and client lists.\n    The Receiving Party shall not disclose, publish, or use this information for any purpose other than evaluating the proposed business relationship without prior written consent.\n    This obligation does not apply to information that is publicly known through no fault of the Receiving Party.\n    Upon termination of discussions, the Receiving Party shall return or destroy all confidential materials.',
               'role': 'assistant',
               'name': 'UserProxyAgent'},
              {'content': 'Here are the key clauses extracted from the provided text:\n\n- Confidentiality obligation\n  - Receiving Party must treat all non-public information from the Disclosing Party

In [20]:
Final_reviewer.chat_messages_for_summary(User_proxy)
# this function returns the conversation held between two agents

[{'content': 'The Receiving Party agrees to treat all non-public information received\n    from the Disclosing Party as strictly confidential. This includes technical data, business plans, and client lists.\n    The Receiving Party shall not disclose, publish, or use this information for any purpose other than evaluating the proposed business relationship without prior written consent.\n    This obligation does not apply to information that is publicly known through no fault of the Receiving Party.\n    Upon termination of discussions, the Receiving Party shall return or destroy all confidential materials.\nContext: \nHere are the key clauses extracted from the provided text:\n\n- Confidentiality obligation\n  - Receiving Party must treat all non-public information from the Disclosing Party as strictly confidential.\n\n- Definition / examples of Confidential Information\n  - Includes technical data, business plans, and client lists (i.e., non-public information).\n\n- Use limitation\n 

In [14]:
dir(chats)

['__add__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

In [17]:
dir(chat_flow)

['__add__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']